# Data Ingestion

Ok, so today we'll be working with the transactions, product, and hh_demographic tables in the project_data folder.

* First, we are reading in the transactions data, bringing in only the columns `household_key`, `BASKET_ID`, `DAY`, `PRODUCT_ID`, `QUANTITY`, and `SALES_VALUE`.

* We then convert `DAY`, `QUANTITY`, and `PRODUCT_ID` to the smallest appropriate integer types, upon ivestigation and assuming this is going to be a static dataset.


In [2]:
import pandas as pd
import numpy as np

In [80]:
transactions = pd.read_csv(
    "project_transactions.csv",
    usecols = ["household_key", "BASKET_ID", "DAY", "PRODUCT_ID", "QUANTITY", "SALES_VALUE"],
    #parse_dates = [""],
    dtype = {
        "household_key": "int16",
        "DAY": "int16",
        "PRODUCT_ID": "int32",
        "QUANTITY": "int32",
        "SALES_VALUE": "float32"
    }
).rename(columns = {'BASKET_ID': 'basket_id', 'DAY':'day', 'PRODUCT_ID': 'product_id', 'QUANTITY': 'quantity', 'SALES_VALUE':'sales_value'})

transactions.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2146311 entries, 0 to 2146310
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   household_key  int16  
 1   basket_id      int64  
 2   day            int16  
 3   product_id     int32  
 4   quantity       int32  
 5   sales_value    float32
dtypes: float32(1), int16(2), int32(2), int64(1)
memory usage: 49.1 MB


In [86]:
transactions = (
    transactions
    # Creating 2016-01-01 and appending the time delta according to the previous 'day' column
    .assign(date = (pd.to_datetime("2016", format='%Y') 
                    + pd.to_timedelta(transactions["day"].sub(1).astype(str) + " days"))
           )
    .drop(["day"], axis=1)
)

In [98]:
transactions.head()

,household_key,basket_id,product_id,quantity,sales_value,date
0,1364,26984896261,842930,1,2.19,2016-01-01
1,1364,26984896261,897044,1,2.99,2016-01-01
2,1364,26984896261,920955,1,3.09,2016-01-01
3,1364,26984896261,937406,1,2.50,2016-01-01
4,1364,26984896261,981760,1,0.60,2016-01-01


## TIME BASED ANALYSIS

* How are sales going over time ? Are sales growing over time?
* Management is particularly interested in the dates from April 2016 and October 2017. Any particular trend in this period ?
* How was 2017 compared to 2016 ?
* How is the day of the week affecting the sales ?
  

# DEMOGRAPHICS

* Read in the `hh_demographic.csv` file, but only the columns `AGE_DESC`, `INCOME_DESC`, `household_key`, and `HH_COMP_DESC`. Convert the appropriate columns to the category dtype.


* Then group the transactions table by household_id, and calculate the sum of SALES VALUE by household.


* Once you've done that, join the demographics DataFrame to the aggregated transactions table. Since we're interested in analyzing the demographic data we have, make sure not to include rows from transactions that don't match.


* Plot the sum of sales by age_desc and income_desc (in separate charts).


* Then, create a pivot table of the mean household sales by `AGE_DESC` and `HH_COMP_DESC`. Which of our demographics have the highest average sales?


# PRODUCT DEMOGRAPHICS

* Read in the product csv file.

* Only read in product_id and department from product (consider converting columns).

* Join the product DataFrame to transactions and demographics tables, performing an inner join when joining both tables.

* Finally, pivot the fully joined dataframe by AGE_DESC and DEPARTMENT, calculating the sum of sales. Which category does our youngest demographic perform well in?



# EXPORT

Finally, export your pivot table to an excel file. Make sure to provide a sheet name.